In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
%matplotlib inline

In [2]:
from lightgbm import LGBMClassifier

In [3]:
import seaborn as sns
sns.set()

In [4]:
import warnings
warnings.filterwarnings("ignore")

In [50]:
from sklearn.preprocessing import LabelEncoder
# train data 입력시 id, target 모두 넣어주어야 한다. 
# test data 입력시 id 모두 넣어주어야 한다.
# from sklearn.preprocessing import LabelEncoder 해주어야 함
def submit_kaggle(df_train, df_test, model, path, user_id = "id", target = "country_destination"):
    le = LabelEncoder()

    # feature, target 전처리
    y_train = le.fit_transform(df_train[target])
    X_train = df_train.drop([target, user_id], axis = 1)

    X_test_id = df_test[user_id] # id 따로 정의
    X_test = df_test.drop([user_id], axis = 1)

    # train data set으로 fitting
    print("model fitting 시작 !!")
    model = model.fit(X_train, y_train)
    predic_proba = model.predict_proba(X_test)
    print("model fitting 완료 !!")
    
    # submit data, predict data 생성
    df_submit = pd.DataFrame(columns=["id", "country"])

    ids = []
    cts = []
    for i in range(len(X_test_id)):
        idx = X_test_id.iloc[i]
        ids += [idx] * 5
        cts += le.inverse_transform(np.argsort(predic_proba[i])[::-1])[:5].tolist()

    df_submit["id"] = ids
    df_submit["country"] = cts
    
    df_submit.to_csv(path, index = False)
    print("csv file 제작 완료 !!")
    
    !kaggle competitions submit -c airbnb-recruiting-new-user-bookings -f {path} -m "Message"

In [52]:
df_train = pd.read_csv("../data/train_age_action_count.csv")
df_test = pd.read_csv("../data/test_age_action_count.csv")
df_session = pd.read_csv("../data/sessions.csv")

In [15]:
df_train_origin = pd.read_csv("../data/train_users_2.csv")
df_test_origin = pd.read_csv("../data/test_users.csv")
df_all = pd.concat([df_train_origin, df_test_origin])

In [17]:
df_all["date_account_created"] = pd.to_datetime(df_all["date_account_created"], format = "%Y-%m-%d")
df_all["timestamp_first_active"] = pd.to_datetime(df_all["timestamp_first_active"], format="%Y%m%d%H%M%S", errors='ignore')

In [22]:
df_all["lag_fa_ac"] = df_all["timestamp_first_active"] - df_all["date_account_created"]

In [46]:
s_train_lag = df_all["lag_fa_ac"].iloc[:213451]
s_test_lag = df_all["lag_fa_ac"].iloc[213451:]

In [47]:
df_train_lag = pd.DataFrame(columns = ["lag_days", "lag_seconds"])
df_test_lag = pd.DataFrame(columns = ["lag_days", "lag_seconds"])

df_train_lag["lag_days"] = s_train_lag.apply(lambda x : -1 * x.days)
df_test_lag["lag_days"] = s_train_lag.apply(lambda x : -1 * x.days)

df_train_lag["lag_seconds"] = s_train_lag.apply(lambda x : x.seconds)
df_test_lag["lag_seconds"] = s_train_lag.apply(lambda x : x.seconds)

In [7]:
lgb = LGBMClassifier(max_depth=4 , n_jobs=4)

In [55]:
df_train["lag_days"] = df_train_lag["lag_days"]
df_test["lag_days"] = df_test_lag["lag_days"]

df_train["lag_seconds"] = df_train_lag["lag_seconds"]
df_test["lag_seconds"] = df_test_lag["lag_seconds"]

In [57]:
df_train.shape, df_test.shape

((213451, 519), (62096, 518))

### lag_days + lag_seconds 더했을 때

0.87693

In [58]:
submit_kaggle(df_train, df_test, lgb, "submit/submit.csv")

model fitting 시작 !!
model fitting 완료 !!
csv file 제작 완료 !!
Successfully submitted to Airbnb New User Bookings